In [ ]:
!pip install -U tqdm --quiet

In [ ]:
import os
import sys
import requests
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from dotenv import load_dotenv

sys.path.append(os.path.join("..", ".."))
from utils.s3_bucket import S3Bucket

In [ ]:
load_dotenv(os.path.join("..", "..", "env"))

In [ ]:
bucket_name = "ava-cv-raw-photo-bucket"
input_prefix = os.path.join("GBIF-csv", "Pl@ntNet")
output_prefix = os.path.join("Pl@ntNet", "plants", "images")

In [ ]:
bucket = S3Bucket(
    bucket_name=bucket_name,
    region_name=os.environ["REGION_NAME"],
)

input_key = os.path.join(input_prefix, "occurrence.txt")
occurrences = pd.read_csv(bucket[input_key], sep="\t")
not_null_columns = occurrences.columns[~occurrences.isnull().all()]
print("Columns that are not null:")
print(list(not_null_columns))

In [ ]:
# Remove irrelevant columns
occurrences = occurrences[["gbifID", "species", "genus"]]
occurrences

In [ ]:
name_mapping = {
    "fragaria": "strawberry",
    "mentha": "mint",
    "origanum vulgare": "oregano",
    "solanum lycopersicum": "tomato",
    "capsicum annuum": "red pepper",
    "cucurbita pepo": "zucchini",
    "coriandrum sativum": "cilantro",
    "anethum graveolens": "dill",
    "lactuca sativa": "lettuce",
    "ocimum basilicum": "basil"
}


def get_name(row):
    genus_to_classify = ["fragaria", "mentha"]
    if row["genus"] in genus_to_classify:
        scientific_name = row["genus"]
    else:
        scientific_name = row["species"]
        
    if scientific_name in name_mapping.keys():
        common_name = name_mapping[scientific_name]
        return common_name
    
    return "nan"


# Create new name column which is a combination of both genus and species names
occurrences[["genus", "species"]] = (
    occurrences[["genus", "species"]]
    .apply(lambda x: x.astype(str).str.lower())
)
occurrences["name"] = occurrences.apply(lambda row: get_name(row), axis=1)
occurrences = occurrences.drop(columns=["species", "genus"])
occurrences = occurrences[occurrences["name"] != "nan"]  # remove any rows whose name is "nan"
occurrences

In [ ]:
occurrences = occurrences[occurrences["name"].isin(["red pepper", "strawberry", "tomato"])]
occurrences

In [ ]:
input_key = os.path.join(input_prefix, "multimedia.txt")
sources = pd.read_csv(bucket[input_key], sep="\t")
sources

In [ ]:
# Get source URL and title for each ID
input_key = os.path.join(input_prefix, "multimedia.txt")
sources = pd.read_csv(bucket[input_key], sep="\t")
sources = sources[["gbifID", "title", "identifier"]]
occurrences = occurrences.merge(sources, how="left", on="gbifID")
occurrences

In [ ]:
# Get image type (leaf, flower, fruit, habit, bark, other)
occurrences["type"] = occurrences["title"].str.replace(r"^.+:([^:]+)$", r"\1", regex=True).str.strip()
occurrences = occurrences.drop(columns=["title"])
occurrences

In [ ]:
occurrences["type"].value_counts()

In [ ]:
occurrences = occurrences[occurrences["type"] == "leaf"]
occurrences = occurrences.drop(columns=["type"])
occurrences

In [ ]:
occurrences["name"].value_counts()

In [ ]:
# Get n random rows from each plant name
n = min(occurrences["name"].value_counts())
occurrences = occurrences.groupby("name").sample(n).reset_index(drop=True)
occurrences["index"] = occurrences.groupby("name").cumcount()  # increasing index per group
occurrences["name"].value_counts()

In [ ]:
def download_image(row, prefix):
    try:
        image = Image.open(requests.get(row["identifier"], stream=True).raw).convert('RGB')
        output_key = os.path.join(prefix, row["name"], f'{row["index"]}-{row["gbifID"]}.jpg')
        bucket[output_key] = image
    except KeyboardInterrupt as exc:
        raise exc
    except:
        print(f'Encountered error when attempting to download {row["identifier"]}')

tqdm.pandas()

In [ ]:
%%time
_ = occurrences.progress_apply(lambda row: download_image(row, output_prefix), axis=1)